In [15]:
import pandas as pd
import tqdm 

In [81]:
filename = "J20250829142736"
data = pd.read_csv(f"../data/{filename}.csv")
data.head()

,年度番号,事故発生日,品名,型式機種,事業者名,被害の種類,事故通知内容,製品の使用期間,事故原因,事故原因区分,再発防止措置,事故通知者,事故受付日,品目
0,2003-0835,2003/08/09,キックスケーター,ＳＰＥＥＤ,（株）サンスジャパン,5.製品破損,購入後、２～３回走行したところ、デッキ部とハンドル部の接合部分が外れた。,約1月,溶接のスピードが速すぎたため、溶け込み不良で強度が不足し、溶接部が外れたものと推定される。,Ａ２,平成１５年１０月輸入分から、溶接の作業時間を延長して接合部分を強化し、輸出前の検品を強化し...,消費者センター,2003/09/04,08 レジャー用品
1,2003-0832,2003/08/26,草刈機,NaN,NaN,1.死亡,植林地で草刈り機を使って草刈り作業をしていた男性が、右大腿部を切断して死亡した。,不明,警察の調査では、被害者が急斜面で草刈り機を使用中にバランスを崩し、誤って右大腿部を切断した...,Ｅ２,製造業者等は不明であり、被害者の不注意とみられる事故であるため、特に措置はとらなかった。,製品評価技術基盤機構,2003/09/04,04 家具・住宅用品
2,2003-0831,2003/09/01,ガスこんろ（都市ガス用）,NaN,NaN,3.軽傷,飲食店で、ラードを加熱していた鍋から火が上がった。消火作業中に天井から換気扇が落下し、１名...,不明,消防の調査では、店員が調理中ガスこんろの火をつけたままその場を離れたため、鍋で加熱していた...,Ｅ２,製造業者等は不明であり、使用者の不注意とみられる事故であるため、措置はとらなかった。,製品評価技術基盤機構,2003/09/04,03 燃焼器具
3,2003-0827,2003/05/12,介護ベッド（電動式）【その他の電気機械器具付家具】,キューマアウラベッドＫＱ－６０２,パラマウントベッド（株）,2.重傷,使用者が手元スイッチの背上げボタンを押していたところ、ボタンから指を放してもベッドが作動し...,約4年,繰返しのスイッチ操作により手指の分泌物・ハンドクリーム等の成分がスイッチのボタン部から内部...,Ａ１,平成１５年８月２７日付けの新聞に社告を掲載し、取引先への通知並びにインターネットにおける情...,製造事業者,2003/09/03,01 家庭用電気製品
4,2003-0826,2003/05/17,介護ベッド（電動式）【その他の電気機械器具付家具】,キューマアウラベッドＫＱ－６０２,パラマウントベッド（株）,2.重傷,電動ベッドを使用中に、手元スイッチを操作していない状態でベッドが作動し、背ボトムと膝ボトム...,約4年,繰返しのスイッチ操作により手指の分泌物・ハンドクリーム等の成分がスイッチのボタン部から内部...,Ａ１,平成１５年８月２７日付けの新聞に社告を掲載し、取引先への通知並びにインターネットにおける情...,製造事業者,2003/09/03,01 家庭用電気製品


In [82]:
import unicodedata
def normalize_text(text):
    text = unicodedata.normalize("NFKC", text)
    return text

In [83]:
def normalize_dataframe(df):
    """
    DataFrameの各セルに対してUnicode正規化（NFKC）を適用する
    
    Args:
        df (pd.DataFrame): 正規化対象のDataFrame
    
    Returns:
        pd.DataFrame: 正規化されたDataFrame
    """
    # DataFrameをコピーして元データを保持
    normalized_df = df.copy()
    
    # 各列に対して処理
    for column in tqdm.tqdm(normalized_df.columns):
        # 文字列型の列のみ処理
        if normalized_df[column].dtype == 'object':
            normalized_df[column] = normalized_df[column].apply(
                lambda x: normalize_text(str(x)) if pd.notna(x) else x
            )
    
    return normalized_df

# データの正規化を実行
normalized_data = normalize_dataframe(data)
print("正規化完了")

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:02<00:00,  6.10it/s]

正規化完了


In [84]:
import re 

# 品名から括弧を除去し、余分な空白も削除
def clean_product_name(text):
    if pd.isna(text):
        return text
    
    # 文字列に変換
    text = str(text)
    
    # 丸括弧を削除
    text = re.sub(r"\(.*?\)", "", text)
    
    # 角括弧を削除
    text = re.sub(r"【.*?】", "", text)
    
    # 余分な空白を削除（連続する空白を単一の空白に変換し、前後の空白を削除）
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# 品目列を作成（括弧除去処理）
normalized_data["品名"] = normalized_data["品名"].apply(clean_product_name)

print("処理完了")

処理完了


In [74]:
# 正規化されたデータを新しいCSVファイルとして保存
output_filename = f"{filename}_normalized.csv"
normalized_data.to_csv(f"../data/{output_filename}", index=False, encoding='utf-8')
print(f"保存しました: {output_filename}")

保存しました: J20250829142736_normalized.csv


In [89]:
normalized_data.head()

,事故発生日,品名,被害の種類,事故通知内容,製品の使用期間,事故原因,事故原因区分,再発防止措置,品目
0,2003/08/09,キックスケーター,[製品破損],購入後、2~3回走行したところ、デッキ部とハンドル部の接合部分が外れた。,約1月,溶接のスピードが速すぎたため、溶け込み不良で強度が不足し、溶接部が外れたものと推定される。,A2,平成15年10月輸入分から、溶接の作業時間を延長して接合部分を強化し、輸出前の検品を強化し...,08 レジャー用品
1,2003/08/26,草刈機,[死亡],植林地で草刈り機を使って草刈り作業をしていた男性が、右大腿部を切断して死亡した。,不明,警察の調査では、被害者が急斜面で草刈り機を使用中にバランスを崩し、誤って右大腿部を切断した...,E2,製造業者等は不明であり、被害者の不注意とみられる事故であるため、特に措置はとらなかった。,04 家具・住宅用品
2,2003/09/01,ガスこんろ,[軽傷],飲食店で、ラードを加熱していた鍋から火が上がった。消火作業中に天井から換気扇が落下し、1名...,不明,消防の調査では、店員が調理中ガスこんろの火をつけたままその場を離れたため、鍋で加熱していた...,E2,製造業者等は不明であり、使用者の不注意とみられる事故であるため、措置はとらなかった。,03 燃焼器具
3,2003/05/12,介護ベッド,[重傷],使用者が手元スイッチの背上げボタンを押していたところ、ボタンから指を放してもベッドが作動し...,約4年,繰返しのスイッチ操作により手指の分泌物・ハンドクリーム等の成分がスイッチのボタン部から内部...,A1,平成15年8月27日付けの新聞に社告を掲載し、取引先への通知並びにインターネットにおける情...,01 家庭用電気製品
4,2003/05/17,介護ベッド,[重傷],電動ベッドを使用中に、手元スイッチを操作していない状態でベッドが作動し、背ボトムと膝ボトム...,約4年,繰返しのスイッチ操作により手指の分泌物・ハンドクリーム等の成分がスイッチのボタン部から内部...,A1,平成15年8月27日付けの新聞に社告を掲載し、取引先への通知並びにインターネットにおける情...,01 家庭用電気製品


In [76]:
normalized_data["品名"].value_counts()

品名
ガスこんろ      2547
石油ストーブ     1841
電気ストーブ     1837
ACアダプター    1686
洗面化粧台      1505
           ... 
末端閉止弁         1
乾燥剤           1
レンジ台          1
灯油窯           1
ガス用配管継手       1
Name: count, Length: 1247, dtype: int64

In [85]:
drop_cols = ["年度番号", "型式機種", "事業者名", "事故通知者", "事故受付日"]

In [86]:
normalized_data = normalized_data.drop(columns=drop_cols)

In [ ]:
# カテゴリを配列として抽出する関数
def extract_damage_categories(text):
    """
    被害の種類から、すべてのカテゴリを配列として抽出
    例: "死亡:11.火災:12.CO中毒" -> ["死亡", "火災", "CO中毒"]
    例: "1.死亡" -> ["死亡"]
    """
    if pd.isna(text):
        return []
    
    # 文字列に変換
    text = str(text)

    # カテゴリを分割
    parts = text.split(':')

    categories = []
    for part in parts:
        part = part.strip()
        # カテゴリを取得
        if '.' in part:
            category = part.split('.', 1)[1].strip()
            # 重複回避
            if category and category not in categories:
                categories.append(category)
        else:
            # ピリオドがない場合はそのまま追加（既に処理済みの可能性）
            if part and part not in categories:
                categories.append(part.strip())
    
    return categories

# すべてのカテゴリを配列として抽出
if "被害の種類" in normalized_data.columns:
    normalized_data["被害の種類"] = normalized_data["被害の種類"].apply(extract_damage_categories)
    print("処理完了")    
else:
    print("エラー: '被害の種類'列が見つかりません")

処理完了


In [88]:
normalized_data["被害の種類"].value_counts()

被害の種類
[製品破損]                17228
[拡大被害]                 8605
[軽傷]                   7729
[火災]                   2102
[重傷]                   1300
[死亡]                    931
[被害なし]                  732
[CO中毒]                   43
[死亡, 火災]                 42
[重傷, 火災]                 25
[死亡, CO中毒]               19
[死亡, 火災, CO中毒]            5
[死亡, 重傷, 火災]              4
[死亡, 重傷, CO中毒]            2
[死亡, 重傷, 火災, CO中毒]        2
[火災, CO中毒]                2
Name: count, dtype: int64

In [91]:
def extract_product_categories(text):

    if pd.isna(text):
        return []
    
    # 文字列に変換
    text = str(text)

    # カテゴリを取得
    if ' ' in text:
        category = text.split(' ', 1)[1].strip()

    return category

if "品目" in normalized_data.columns:
    normalized_data["品目"] = normalized_data["品目"].apply(extract_product_categories)
    print("処理完了")    
else:
    print("エラー: '品目'列が見つかりません")

処理完了


In [92]:
normalized_data["品目"].value_counts()

品目
家庭用電気製品    17146
燃焼器具        8911
家具・住宅用品     3875
身のまわり品      3470
乗物・乗物用品     1628
乳幼児用品       1396
台所・食卓用品      871
レジャー用品       688
保健衛生用品       434
繊維製品         319
その他           33
Name: count, dtype: int64

In [93]:
# CSVファイルとして保存
normalized_data.to_csv(f"../data/{filename}_processed.csv", index=False, encoding='utf-8')
print(f"保存しました: {filename}_processed.csv")

保存しました: J20250829142736_processed.csv
